In [1]:
import pandas as pd
import config


## Exploring CSV

In [2]:
csv_path='/Users/taoluo/Workspace/lorne/agltest/coding_practice_python_battery_dispatch_dataset.csv'

# read header from 1st line
df = pd.read_csv(config.CSV_PATH, header=0)

df.head()

,Timestamp,INITIALMW,TARGETMW,RRP
0,1/04/2024 0:05,0.0,0.0,59.66236
1,1/04/2024 0:10,0.0,0.0,59.75000
2,1/04/2024 0:15,0.0,0.0,59.20843
3,1/04/2024 0:20,0.0,0.0,59.59650
4,1/04/2024 0:25,0.0,0.0,58.49123


## Function to calculate interval revenue

### Asumption:
- As mentioned, INITIALMW and TARGETMW in each interval are linear ramping 
- intervals are fixed at 5 mins, so it hard coded below
- Discharging(positive MW) bring revenue, charging(negative MW) have no revenue but also no cost, this follows solar panel's battery

In [10]:
def calc_interval_revenue(initial, target, unit_price):
    """calculate revenue in 5mins by initial and target"""
    # the ramp are under 0, indicate it's charing in whole interval, no revenue generated
    if initial <= 0 and target <= 0:
        return 0

    if unit_price == 0:
        return 0

    # the ramp above 0, calculate as trapezoid area
    if initial >= 0 and target >= 0:
        # (initial + target) / 12 / 2
        # / 12 for MWh in 5 mins
        # / 2 for  trapezoid area formula
        return (initial + target) / 24 * unit_price

    # the ramp across zero, only calculate the positive area
    charge_rate = max(initial, target)
    charge_ratio = abs(charge_rate) / (abs(initial) + abs(target))
    return charge_rate / 24 * charge_ratio * unit_price
    

## Apply to Pandas 

Calculate revenue for each interval

In [6]:
calc_revenue = lambda row: calc_interval_revenue(row['INITIALMW'],row['TARGETMW'],row['RRP'])
df['revenue'] = df.apply(calc_revenue, axis=1)
df.tail(100)

,Timestamp,INITIALMW,TARGETMW,RRP,revenue
188,1/04/2024 15:45,0.00000,0.0,60.66444,0.000000
189,1/04/2024 15:50,1.13012,0.0,85.27446,4.015432
190,1/04/2024 15:55,-0.08866,-8.0,77.88982,0.000000
191,1/04/2024 16:00,-10.17502,0.0,63.17193,0.000000
192,1/04/2024 16:05,0.00000,0.0,49.99301,0.000000
...,...,...,...,...,...
283,1/04/2024 23:40,-0.09306,0.0,65.90012,0.000000
284,1/04/2024 23:45,-0.09306,0.0,59.75000,0.000000
285,1/04/2024 23:50,-0.07211,0.0,59.55000,0.000000
286,1/04/2024 23:55,0.90866,0.0,61.92004,2.344344


## Get total revenue

In [9]:
total_revenue = sum(df['revenue'])
print(f'The total revenue in 1 Apr 2024 is ${total_revenue:.2f}')

The total revenue in 1 Apr 2024 is $8530.08
